# **Assignment 3: Vectorized Computations using TensorFlow**

1. Create a matrix U of shape (m, n) with input values where m and n are input positive integers.


In [37]:
import tensorflow as tf
m = 2 # number of examples
n = 3 # input features
p = 3 # hidden layer units
k = 10
U = tf.constant([[1, 2, 3], [4, 5, 6]], dtype=tf.float32)
print(U)

tf.Tensor(
[[1. 2. 3.]
 [4. 5. 6.]], shape=(2, 3), dtype=float32)


2. Compute X as the transpose of U .


In [40]:
X = tf.transpose(U)
display(X)

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[1., 4.],
       [2., 5.],
       [3., 6.]], dtype=float32)>

3. Create a matrix Y of shape (1, m) with random integer values ∈ [0, 9].

In [45]:
Y = tf.random.uniform(shape=(1, m), minval=0, maxval=10, dtype=tf.int32)
print(Y)

tf.Tensor([[6 2]], shape=(1, 2), dtype=int32)


4. Create a matrix W 1 of shape (p, n) with random values ∈ [0, 1] where p is an input positive integer.

In [46]:
W1=tf.Variable(tf.random.uniform((p, n), 0, 1))
print(W1)

<tf.Variable 'Variable:0' shape=(3, 3) dtype=float32, numpy=
array([[0.17167163, 0.34061563, 0.6338539 ],
       [0.37236524, 0.06973135, 0.23149478],
       [0.00828755, 0.0759238 , 0.5779438 ]], dtype=float32)>


5. Create a vector B1 of shape (p, 1) with random values ∈ [0,1].

In [50]:
B1 =  tf.Variable(tf.random.uniform((p, 1), 0, 1))
print(B1)

<tf.Variable 'Variable:0' shape=(3, 1) dtype=float32, numpy=
array([[0.05330014],
       [0.25209427],
       [0.31675577]], dtype=float32)>


6. Create a matrix W 2 of shape (10, p) with all zeros.

In [51]:
W2 = tf.Variable(np.zeros((10, p)), dtype=tf.float32)
display(W2)

<tf.Variable 'Variable:0' shape=(10, 3) dtype=float32, numpy=
array([[0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]], dtype=float32)>

7. Create a scalar B2 with a random value ∈ [0, 1].

In [52]:
B2 =  tf.Variable(tf.random.uniform((k, 1), 0, 1))
print(B2)

<tf.Variable 'Variable:0' shape=(10, 1) dtype=float32, numpy=
array([[0.8634516 ],
       [0.54351056],
       [0.872321  ],
       [0.93135536],
       [0.47907603],
       [0.48532975],
       [0.747301  ],
       [0.54385877],
       [0.32134843],
       [0.1691283 ]], dtype=float32)>


8. Perform the following computations iteratively for 15 times:

(a) \[ Z1 = W1 \cdot X + B1 \]

(b) A1 = ReLU (Z1) where ReLU (x) is a function that returns 0 for negative values and the input value itself otherwise.

(c) \[ Z2 = W2 \cdot A1 + B2 \]

(d) A2 = softmax(Z2) where
$
\text{softmax}(x) = \frac{e^{x_i}}{\sum_j e^{x_j}}
$

(e) dZ2 = A2 − one hot Y where one hot Y is the one-hot encoded form of Y .

(f) $ dA2 = W2^T·dZ2$

(g) $ dW 2 = \frac{1}{m} · dZ2 · A1^T $

(h) $ dB2 = \frac{1}{m} \cdot \sum dZ2 $ (sum along the columns)

(i) $ dZ1 = dA2 \circ \text{ReLU}'(Z1) $ where ReLU'(x) returns 1 for positive values and 0 otherwise, and ◦ indicates element-wise multiplication.

(j) $ dA1 = W1^T \cdot dZ1 $

(k) $ dB1 = \frac{1}{m} \cdot \sum dZ1 $ (sum along the columns)

(l) $ dW1 = \frac{1}{m} \cdot dZ1 \cdot X^T $

(m) Update and print W 1, B1, W 2, and B2 for α = 0.01:


* i) W1 = W1 − α · dW1
* ii) B1 = B1 − α · dB1
* iii) W2 = W2 − α · dW2
* iv) B2 = B2 − α · dB2



In [54]:
def relu(z):
  return tf.maximum(0.0, z)
def relu_prime(z):
  return tf.cast(z > 0, tf.float32)
def softmax(z):
  expz = tf.exp(z - tf.reduce_max(z, axis=1, keepdims=True))
  return expz / tf.reduce_sum(expz, axis=1, keepdims=True)
alpha = tf.constant(0.01)
num_iters = 15
for i in range(num_iters):
  Z1 = tf.matmul(W1, X) + B1
  A1 = relu(Z1)
  Z2 = tf.matmul(W2, A1) + B2
  Z2 = tf.transpose(Z2)
  A2 = softmax(Z2)
  Y_reshaped = tf.reshape(Y, [-1])
  Y_onehot = tf.one_hot(Y_reshaped, depth=k)
  dZ2 = A2 - Y_onehot
  dW2 = (1. / tf.cast(m, tf.float32)) * tf.matmul(tf.transpose(dZ2), tf.transpose(A1))
  dB2 = (1. / tf.cast(m, tf.float32)) * tf.reduce_sum(tf.transpose(dZ2),axis=1, keepdims=True)
  dA1 = tf.matmul(tf.transpose(W2), tf.transpose(dZ2))
  dZ1 = dA1 * relu_prime(Z1)
  dW1 = (1. / tf.cast(m, tf.float32)) * tf.matmul(dZ1, tf.transpose(X))
  dB1 = (1. / tf.cast(m, tf.float32)) * tf.reduce_sum(dZ1, axis=1,keepdims=True)
  W1.assign_sub(alpha * dW1)
  B1.assign_sub(alpha * dB1)
  W2.assign_sub(alpha * dW2)
  B2.assign_sub(alpha * dB2)
  tf.print("\nIteration", i + 1)
  tf.print("W1:\n", W1)
  tf.print("B1:\n", B1)
  tf.print("W2:\n", W2)
  tf.print("B2:\n", B2)


Iteration 1
W1:
 [[0.171671629 0.34061563 0.633853912]
 [0.372365236 0.0697313547 0.231494784]
 [0.00828754902 0.0759238 0.577943802]]
B1:
 [[0.0533001423]
 [0.252094269]
 [0.316755772]]
W2:
 [[-0.0057528452 -0.00313732307 -0.00407156069]
 [-0.00417767 -0.00227829884 -0.00295673427]
 [0.0254268423 0.0142306285 0.0168781057]
 ...
 [-0.00417912519 -0.00227909232 -0.00295776431]
 [-0.00334541756 -0.00182442868 -0.00236771023]
 [-0.00287304213 -0.00156681798 -0.00203338778]]
B2:
 [[0.862180829]
 [0.542587698]
 [0.876038909]
 ...
 [0.54293561]
 [0.320609421]
 [0.168493643]]

Iteration 2
W1:
 [[0.172174796 0.341272712 0.634664893]
 [0.372644722 0.0700946599 0.231941909]
 [0.00862802 0.0763736069 0.578502953]]
B1:
 [[0.0534540638]
 [0.252178073]
 [0.316865116]]
W2:
 [[-0.0111299725 -0.00606779847 -0.00788319297]
 [-0.00814099051 -0.00443856791 -0.00576524483]
 [0.0493866913 0.0276533142 0.0327422619]
 ...
 [-0.00814377237 -0.0044400841 -0.00576721551]
 [-0.00654424401 -0.00356812729 -0.00463